In [4]:
!pip install torchvision
!pip install kaggle kagglehub
!pip install h5py

In [5]:
import os
import numpy as np
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import h5py

AttributeError: partially initialized module 'torchvision' has no attribute 'extension' (most likely due to a circular import)

In [16]:
path = kagglehub.dataset_download("sudipde25/mri-dataset-for-detection-and-analysis")
print("Path to dataset files:", path)

#mat_data = scio.loadmat('')

# images_numpy = mat_data['images']
# labels_numpy = mat_data['labels']

Path to dataset files: /ccsopen/home/dom_p/.cache/kagglehub/datasets/sudipde25/mri-dataset-for-detection-and-analysis/versions/20


In [69]:
class MatFileDataset(Dataset):
    def __init__(self, mat_dir, transform=None):
        self.mat_dir = mat_dir
        self.mat_files = [f for f in os.listdir(mat_dir) if f.endswith('.mat')]
        self.transform = transform

    def __len__(self):
        return len(self.mat_files)

    def __getitem__(self, idx):
        mat_path = os.path.join(self.mat_dir, self.mat_files[idx])
        
        with h5py.File(mat_path, 'r') as mat_data:
            cvind_data = mat_data['cvind']
            print(cvind_data[0])
            # TODO: This DOESN'T work. Figure out how to parse cvind data.
            image_raw = cvind_data[idx]['image'][()]
            print(image_raw)
            label = cvind_data[idx]['label'][()]
            print(label)

        image = image_raw.astype(np.float32)

        if self.transform:
            image = self.transform(image)

        # Add channel dimension if image is grayscale (assuming images are 2D)
        if image.ndim == 2:
            image = np.expand_dims(image, axis=0)

        return torch.tensor(image), torch.tensor(label)

In [70]:
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((128, 128)),  # Resize to desired dimensions
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])  # Normalize for grayscale images
])

# Create Dataset and Dataloader
dataset = MatFileDataset(path, transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [71]:
# Define the CNN model
class TumorClassifier(nn.Module):
    def __init__(self):
        super(TumorClassifier, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)  # Adjust input channels to match your data
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(64 * 32 * 32, 128)  # Adjust based on input size after pool
        self.fc2 = nn.Linear(128, 2)  # TODO: Replace with accurate classification, currently binary

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 64 * 32 * 32)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [72]:
# Initialize the model, loss function and optimizer
model = TumorClassifier()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [73]:
# Training loop
num_epochs = 10

for epoch in range(num_epochs):
    for images, labels in dataloader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

# Save the model
torch.save(model.state_dict(), 'tumor_classifier.pth')

print("Training complete. Model saved.")

[5. 5. 5. ... 4. 2. 1.]


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices